In [1]:
import scripts.stock_plots as stock_plots
import scripts.stock_analysis as stock_ana
import plotly.graph_objects as go

from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
def download_data(stocks):
    stocks_str = " ".join(stocks)
    return yf.Tickers(stocks_str)

In [3]:
big = ["SMCI", "AAPL", "GOOG", "AMZN"]
stock_data = download_data(big)

In [4]:
selected = "SMCI"

stock_price = stock_data.tickers[selected].history(period=f"400d")

# stock_price = stock_data.tickers[stock].history(period=f"{past_days}d")

stock_plot = stock_plots.PlotInfo(stock_price)
candle = stock_plot.generate_candle_plot(p2p_order=4)
candle.update_layout(title={"text": selected})



In [15]:
op_dates = stock_data.tickers[selected].options

call_chain = stock_data.tickers[selected].option_chain(op_dates[3]).calls
idx = sum(call_chain["inTheMoney"])

put_chain = stock_data.tickers[selected].option_chain(op_dates[3]).puts
idxp = sum(put_chain["inTheMoney"])

print(call_chain.iloc[idx-1], put_chain.iloc[-idxp])

contractSymbol             SMCI240510C00895000
lastTradeDate        2024-04-12 19:41:01+00:00
strike                                   895.0
lastPrice                                 92.0
bid                                       88.1
ask                                       93.7
change                               -1.099999
percentChange                        -1.181524
volume                                     5.0
openInterest                                 4
impliedVolatility                     0.918519
inTheMoney                                True
contractSize                           REGULAR
currency                                   USD
Name: 16, dtype: object contractSymbol             SMCI240510P00900000
lastTradeDate        2024-04-12 19:41:17+00:00
strike                                   900.0
lastPrice                                89.17
bid                                       87.0
ask                                       90.5
change                              

In [44]:
def std_cal(strike, iv, days):
    std = iv * strike * np.sqrt(days/365)
    return strike+std, strike-std

In [30]:
strike = call_chain.iloc[idx-1]["strike"]
iv = call_chain.iloc[idx-1]["impliedVolatility"]
days = (datetime.strptime(op_dates[1], '%Y-%m-%d') - datetime.now()).days
std_cal(strike, iv, days)

1037.7121778882001 752.2878221117998


In [ ]:
big = ["SMCI", "AAPL", "GOOG", "AMZN"]


In [57]:
up = []
low = []
date = [datetime.now()]
for i in [0,1,2,3,4,5,6]:
    tick = "SMCI"
    call_chain = stock_data.tickers[tick].option_chain(op_dates[i]).calls
    idx = sum(call_chain["inTheMoney"]) - 1
    strike = call_chain.iloc[idx]["strike"]
    iv = call_chain.iloc[idx]["impliedVolatility"]
    days = (datetime.strptime(op_dates[i], '%Y-%m-%d') - datetime.now()).days
    # print(strike, iv, days)
    # std_cal(strike, iv, days)

    put_chain = stock_data.tickers[tick].option_chain(op_dates[i]).puts
    idx1 = -sum(put_chain["inTheMoney"])
    strike1 = put_chain.iloc[idx]["strike"]
    iv1 = put_chain.iloc[idx]["impliedVolatility"]
    # print(strike1, iv1, days)
    # std_cal(strike1, iv1, days)

    strike = (strike + strike1) / 2
    iv = (iv + iv1) / 2
    # print(strike, iv, days)
    # std_cal(strike, iv, days)
    # print("-")

    ans = std_cal(strike, iv, days)

    date.append(op_dates[i])
    if not up:
        up.append(strike)
        low.append(strike)

    up.append(ans[0])
    low.append(ans[1])


In [58]:
# Create line plots
trace1 = go.Scatter(x=date, y=up, mode='lines', name='Line 1')
trace2 = go.Scatter(x=date, y=low, mode='lines', name='Line 2')

# Create figure and add traces
fig = go.Figure([trace1, trace2])

# Update layout
fig.update_layout(
    title='Two Line Plots',
    xaxis=dict(title='X-axis Title'),
    yaxis=dict(title='Y-axis Title')
)

# Show plot
fig.show()

In [60]:
# candle

candle.add_trace(
            go.Scatter(x=date, y=up, mode='lines', name='Line 1'),
            row=1,
            col=1,
        )

TypeError: Figure.add_trace() got multiple values for argument 'row'